# U2AF2 individuals
DEVDFB: [Developmental delay, dysmorphic facies, and brain anomalies](https://omim.org/entry/620535).

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/U2AF2_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Mandibular prognathia,Vertical nystagmus,Chorea,Lower limb hyperreflexia,Thick eyebrow,Upslanted palpebral fissure,Depressed nasal bridge,Deep philtrum,Retrognathia,Delayed CNS myelination
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0000303,HP:0010544,HP:0002072,HP:0002395,HP:0000574,HP:0000582,HP:0005280,HP:0002002,HP:0000278,HP:0002188
1,PMID:34112922,"Global developmental delay, systemic dysmorphism and epilepsy in a patient with a de novo U2AF2 variant",A Japanese girl,NaN,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",NM_007279.3,c.445C>T,na,p.(Arg149Trp),...,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 37 fields


In [5]:
U2AF2_transcript = "NM_007279.3"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="U2AF2",
                          transcript=U2AF2_transcript,
                          allele_1_column_name="allele_1")

[INFO] encoding variant "c.445C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_007279.3%3Ac.445C>T/NM_007279.3?content-type=application%2Fjson
[INFO] encoding variant "c.603G>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_007279.3%3Ac.603G>T/NM_007279.3?content-type=application%2Fjson
[INFO] encoding variant "c.470C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_007279.3%3Ac.470C>T/NM_007279.3?content-type=application%2Fjson


In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,3,"c.445C>T, c.603G>T, c.470C>T"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,3


In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A Japanese girl (FEMALE; P7Y6M),"Developmental delay, dysmorphic facies, and brain anomalies (OMIM:620535)",NM_007279.3:c.445C>T (heterozygous),P3M: Microcephaly (HP:0000252)Single transverse palmar crease (HP:0000954); Pes planus (HP:0001763); Exotropia (HP:0000577); Ptosis (HP:0000508); Global developmental delay (HP:0001263); Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373); Hypoplasia of the corpus callosum (HP:0002079); Hypotonia (HP:0001252); excluded: Autistic behavior (HP:0000729); excluded: Microtia (HP:0008551); excluded: Low-set ears (HP:0000369); excluded: Mandibular prognathia (HP:0000303); excluded: Vertical nystagmus (HP:0010544); excluded: Chorea (HP:0002072); excluded: Lower limb hyperreflexia (HP:0002395); excluded: Thick eyebrow (HP:0000574); excluded: Upslanted palpebral fissure (HP:0000582); excluded: Depressed nasal bridge (HP:0005280); excluded: Deep philtrum (HP:0002002); excluded: Retrognathia (HP:0000278); excluded: Delayed CNS myelination (HP:0002188)
proband (MALE; P6Y),"Developmental delay, dysmorphic facies, and brain anomalies (OMIM:620535)",NM_007279.3:c.603G>T (heterozygous),Microcephaly (HP:0000252); Global developmental delay (HP:0001263); Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373); Hypoplasia of the corpus callosum (HP:0002079); Microtia (HP:0008551); Low-set ears (HP:0000369); Mandibular prognathia (HP:0000303); excluded: Single transverse palmar crease (HP:0000954); excluded: Pes planus (HP:0001763); excluded: Exotropia (HP:0000577); excluded: Ptosis (HP:0000508); excluded: Hypotonia (HP:0001252); excluded: Autistic behavior (HP:0000729); excluded: Vertical nystagmus (HP:0010544); excluded: Chorea (HP:0002072); excluded: Lower limb hyperreflexia (HP:0002395); excluded: Thick eyebrow (HP:0000574); excluded: Upslanted palpebral fissure (HP:0000582); excluded: Depressed nasal bridge (HP:0005280); excluded: Deep philtrum (HP:0002002); excluded: Retrognathia (HP:0000278); excluded: Delayed CNS myelination (HP:0002188)
2-year-old female (FEMALE; P2Y),"Developmental delay, dysmorphic facies, and brain anomalies (OMIM:620535)",NM_007279.3:c.470C>T (heterozygous),Vertical nystagmus (HP:0010544); Global developmental delay (HP:0001263); Hypoplasia of the corpus callosum (HP:0002079); Chorea (HP:0002072); Lower limb hyperreflexia (HP:0002395); Thick eyebrow (HP:0000574); Upslanted palpebral fissure (HP:0000582); Depressed nasal bridge (HP:0005280); Deep philtrum (HP:0002002); Retrognathia (HP:0000278); Delayed CNS myelination (HP:0002188); excluded: Single transverse palmar crease (HP:0000954); excluded: Pes planus (HP:0001763); excluded: Exotropia (HP:0000577); excluded: Ptosis (HP:0000508); excluded: Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373); excluded: Hypotonia (HP:0001252); excluded: Autistic behavior (HP:0000729); excluded: Microtia (HP:0008551); excluded: Low-set ears (HP:0000369); excluded: Mandibular prognathia (HP:0000303)


In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 3 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [11]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = "PMID:34112922"
hpoa_table_creator = builder.autosomal_dominant(PMID).build()

[pyphetools] Ingested 3 GA4GH phenopackets.
We found a total of 22 unique HPO terms
Extracted disease: Developmental delay, dysmorphic facies, and brain anomalies (OMIM:620535)


In [12]:
df = hpoa_table_creator.get_dataframe()
df.head()

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",HP:0000954,Single transverse palmar crease,,,1/1,,,,,PMID:34112922,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",HP:0001763,Pes planus,,,1/1,,,,,PMID:34112922,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",HP:0000577,Exotropia,,,1/1,,,,,PMID:34112922,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",HP:0000508,Ptosis,,,1/1,,,,,PMID:34112922,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620535,"Developmental delay, dysmorphic facies, and brain anomalies",HP:0000252,Microcephaly,,,1/1,,,,,PMID:34112922,PCS,ORCID:0000-0002-0736-9199


In [13]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620535.tab
